# Create Plot

In [ ]:
from __future__ import print_function

from aiida import load_dbenv, is_dbenv_loaded
from aiida.backends import settings
if not is_dbenv_loaded():
    load_dbenv(profile=settings.AIIDADB_PROFILE)

from aiida.orm.querybuilder import QueryBuilder
from aiida.orm.data.structure import StructureData
from aiida.orm.calculation import Calculation
from aiida.orm.data.parameter import ParameterData

import ase.io
import sys
import numpy as np
import ipywidgets as ipw
from base64 import b64decode
from IPython.display import display, clear_output, Image
from fileupload import FileUploadWidget
from tempfile import NamedTemporaryFile
from base64 import b64encode


%matplotlib notebook

import matplotlib
import numpy as np
import matplotlib.pyplot as plt

#import nglview

In [ ]:
def search():
    
    query_message.value = "searching..."
    
    # query AiiDA database
    filters = {}
    
    pk_list = inp_pks.value.strip().split()
    if pk_list:
        filters['id'] = {'in': pk_list}
            
    def add_range_filter(bounds, label):
        filters['attributes.'+label] = {'and':[{'>=':bounds[0]}, {'<':bounds[1]}]}

    add_range_filter(inp_dens.value, "density")
    add_range_filter(inp_del_cap.value, "deliverable_capacity")
    add_range_filter(inp_abs_uptake_hp.value, "absolute_methane_uptake_high_P")
    add_range_filter(inp_abs_uptake_lp.value, "absolute_methane_uptake_low_P")

    
    qb = QueryBuilder()        
    qb.append(ParameterData,
              filters=filters,
              project =
               [
                   'attributes.'+prop_one.value,
                   'attributes.'+prop_two.value,
               ]
             )
#    qb.limit(5)
    #qb.order_by({CifData:{'ctime':'desc'}})

    l = qb.all()
    
    #print(l)
    
    res = map(list, zip(*l))
    list1 = map(float, res[0])
    list2 = map(float, res[1])
    query_message.value = "%d found" % len(res[0])
    plt.plot(list1, list2, 'bo')

In [ ]:
# search UI
style = {"description_width":"220px"}
layout = ipw.Layout(width="90%")
inp_pks = ipw.Text(description='PKs', placeholder='e.g. 4062 4753 (space separated)', layout=layout, style=style)

def slider(desc, min, max):
    return ipw.FloatRangeSlider(description=desc, min=min, max=max, 
                                    value=[min, max], step=0.05, layout=layout, style=style)

inp_dens = slider("Density [kg/m^3]:", 0.0, +1000.0)
inp_del_cap = slider("Del. cap. [v STP/v]:", 0.0, +300.0)
inp_abs_uptake_hp = slider("Abs. meth. uptake high P [mol/kg]:", 0.0, +200.0)
inp_abs_uptake_lp = slider("Abs meth. uptake low P [mol/kg]:",  0.0, +200.0)
search_crit = [inp_pks, inp_dens, inp_del_cap, inp_abs_uptake_hp, inp_abs_uptake_lp]

In [ ]:
def on_click(b):
    with info_out:
        clear_output()
        search()
properties = {
    'Density':'density',
    'Deliverable capacity': 'deliverable_capacity',
    'Methane uptake High P': 'absolute_methane_uptake_high_P',
    'Methane uptake Low P': 'absolute_methane_uptake_low_P'
}
button = ipw.Button(description="Plot")
button.on_click(on_click)
prop_one = ipw.Dropdown(
    options=properties,
    value = 'density',
    description='X:',
)
prop_two = ipw.Dropdown(
    options=properties,
    value='deliverable_capacity',
    description='Y:',
)

properties = ipw.HBox([prop_one, prop_two])

results = ipw.HTML()
info_out = ipw.Output()
query_message = ipw.HTML("press Plot")
app = ipw.VBox(children=search_crit + [properties, button, query_message, results, info_out])
display(app)